### Resumable Downloads
The download scripts above and below are designed to be resumable. If the process is interrupted or an error occurs, simply re-run the cell. It will automatically check for existing files and skip those that have already been successfully downloaded. This applies to planets, satellites, and asteroids.

In [ ]:
# Planets and Dwarf Planets

from astroquery.jplhorizons import Horizons
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, CartesianRepresentation
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy import units as u
import os
import time
from tqdm import tqdm

def convert_xyz_to_ecliptic(x, y, z):
    """ Use AstroPy to convert Cartesian coordinates to ecliptic longitude and latitude """
    # Create a celestial coordinate object, coordinates in astronomical units (AU)
    cart_coord = CartesianRepresentation(x * u.au, y * u.au, z * u.au)
    # Convert Cartesian coordinates to ecliptic coordinate system, specify J2000 reference epoch (default)
    sky_coord = SkyCoord(cart_coord, frame=HeliocentricMeanEcliptic(equinox='J2000'))
    # Return ecliptic longitude and latitude in degrees
    ecl_lon = sky_coord.lon.wrap_at(360 * u.deg).degree  # Normalize to 0-360 degrees
    ecl_lat = sky_coord.lat.degree
    return ecl_lon, ecl_lat

# List of celestial bodies
planets = ['SSB', '199', '299', '399', '499', '599', '699', '799', '899']
# 999 or 134340 Pluto, earliest 1800-JAN-03
dwarfs = ['Ceres', '999', '136108', '136472', '136199','90482','120347','50000','225088','90377']
combined_list = planets + dwarfs

# Define time range
start_date = '1849-01-01'
end_date = '2051-01-01'
# Set time parameters, one data point per day
epochs = {'start': start_date, 'stop': end_date, 'step': '1d'}

# Ensure directories exist
xyz_dir = '../../data/00_raw/helio_cart_states_00h/planets_dwarfs_daily_1849/'
lon_lat_dir = '../../data/00_raw/helio_ecl_sph_00h/planets_dwarfs_daily_1849/'
os.makedirs(xyz_dir, exist_ok=True)
os.makedirs(lon_lat_dir, exist_ok=True)

# Query for each planet
for planet_id in tqdm(combined_list, desc="Processing celestial bodies"):
    # Create Horizons object
    obj = Horizons(id=planet_id, location='@10', epochs=epochs)
    # Get Cartesian positions in heliocentric ecliptic coordinate system
    eph = obj.vectors()
    # Print column names of the returned table to check fields
    # print(eph.colnames)
    
    # Convert to ecliptic coordinates
    eph['ecl_lon'], eph['ecl_lat'] = convert_xyz_to_ecliptic(eph['x'], eph['y'], eph['z'])
    
    df_xyz = pd.DataFrame({
        'date': eph['datetime_str'],  # Date
        'x': eph['x'],   # X component of position vector, AU
        'y': eph['y'],
        'z': eph['z'],
        'vx': eph['vx'],  # X component of velocity vector (au/day)
        'vy': eph['vy'],
        'vz': eph['vz'],
        'lighttime': eph['lighttime'],  # One-way down-leg Newtonian light-time, in days
        'range': eph['range'],  # Distance from coordinate center (au)
        'range_rate': eph['range_rate'] # Radial velocity relative to coordinate center (au/day)
    })
    
    # Create a DataFrame to store ecliptic longitude and latitude data
    df_lon_lat = pd.DataFrame({
        'date': eph['datetime_str'],
        f'{planet_id}_lon': eph['ecl_lon'],
        f'{planet_id}_lat': eph['ecl_lat']
    })
    
    # Save as CSV files
    xyz_filename = f'{xyz_dir}{planet_id}_xyz.csv'
    lon_lat_filename = f'{lon_lat_dir}{planet_id}_lon_lat.csv'
    
    df_xyz.to_csv(xyz_filename, index=False)
    df_lon_lat.to_csv(lon_lat_filename, index=False)
    
    # Add a short delay to avoid sending too many requests to the server
    time.sleep(1)

    print(f'{planet_id} query xyz, convert to ecliptic, finished, file saved')
print('Task completed')

In [ ]:
# Satellites

from astroquery.jplhorizons import Horizons
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, CartesianRepresentation
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy import units as u
import os
import time
from tqdm import tqdm

def get_parent_planet_id(satellite_id):
    """Get parent planet ID based on satellite ID"""
    # Convert to string for processing
    satellite_str = str(satellite_id)
    # Take the first digit + 99
    return satellite_str[0] + '99'

def convert_xyz_to_ecliptic(x, y, z):
    """ Use AstroPy to convert Cartesian coordinates to ecliptic longitude and latitude """
    # Create a celestial coordinate object, coordinates in astronomical units (AU)
    cart_coord = CartesianRepresentation(x * u.au, y * u.au, z * u.au)
    # Convert Cartesian coordinates to ecliptic coordinate system, specify J2000 reference epoch (default)
    sky_coord = SkyCoord(cart_coord, frame=HeliocentricMeanEcliptic(equinox='J2000'))
    # Return ecliptic longitude and latitude in degrees
    ecl_lon = sky_coord.lon.wrap_at(360 * u.deg).degree  # Normalize to 0-360 degrees
    ecl_lat = sky_coord.lat.degree
    return ecl_lon, ecl_lat

# Normal satellite list
s3 = [301]
s4 = [401, 402]
s5 = [
    501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512,
    513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524,
    525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536,
    537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548,
    549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560,
    561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572,
    55501, 55502, 55503, 55504, 55505, 55506, 55507, 55508, 55509,
    55510, 55511, 55512, 55513, 55514, 55515, 55516, 55517, 55518,
    55519, 55520, 55521, 55522, 55523
]
s6 = [
    601, 602, 603, 604, 605, 606, 607, 608, 609, 612, 613, 614,
    619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
    631, 632, 634, 636, 637, 638, 639, 640, 641, 642, 643, 644,
    645, 646, 647, 648, 650, 651, 652, 654, 655, 656, 657, 658,
    659, 660, 661, 662, 663, 664, 665, 666, 65067, 65070, 65077,
    65079, 65081, 65082, 65084, 65085, 65086, 65087, 65088, 65089,
    65090, 65091, 65092, 65093, 65094, 65095, 65096, 65097, 65098,
    65100, 65101, 65102, 65103, 65104, 65105, 65106, 65107, 65108,
    65109, 65110, 65111, 65112, 65113, 65114, 65115, 65116, 65117,
    65118, 65119, 65120, 65121, 65122, 65123, 65124, 65125, 65126,
    65127, 65128, 65129, 65130, 65131, 65132, 65133, 65134, 65135,
    65136, 65137, 65138, 65139, 65140, 65141, 65142, 65143, 65144,
    65145, 65146, 65147, 65148, 65149, 65150, 65151, 65152, 65153,
    65154, 65155, 65156, 65157
]
s7 = [701, 702, 703, 704, 705, 716, 717, 718, 719, 720, 721, 722, 723, 724, 75051]
s8 = [801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 85051, 85052]
s9 = [901, 902, 903, 904, 905]

s0 = s3 + s4 + s5 + s6 + s7 + s8 + s9

# Special satellite list
st635 = [635]
st6 = [610, 611, 615, 616, 617, 618, 633, 649, 653]
st7 = [706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 725, 726, 727]

# Normal date range
s0_date = ['1849-01-01','2051-01-01']

# Special date range
st635_date = ['1990-06-15', '2018-01-16']
st6_date =  ['1949-12-27', '2050-01-01']
st7_date = ['1980-01-02','2051-01-01']

def process_satellite_data(satellite_id, start_date, end_date, xyz_dir, rel_xyz_dir, ecliptic_dir):
    """Process data for a single satellite, including heliocentric and planet-centric coordinates and ecliptic coordinate conversion"""
    try:
        # Set file paths
        xyz_filename = f'{xyz_dir}{satellite_id}_xyz.csv'
        rel_xyz_filename = f'{rel_xyz_dir}{satellite_id}_rel_xyz.csv'
        ecliptic_filename = f'{ecliptic_dir}{satellite_id}_lon_lat.csv'
        
        # If all files already exist, skip processing
        if os.path.exists(xyz_filename) and os.path.exists(rel_xyz_filename) and os.path.exists(ecliptic_filename):
            print(f'All files for satellite {satellite_id} already exist, skipping processing')
            return True
            
        # Set time parameters
        epochs = {'start': start_date, 'stop': end_date, 'step': '1d'}
        
        # Get parent planet ID of the satellite
        parent_id = get_parent_planet_id(satellite_id)
            
        # Create Horizons object - relative to solar center
        helio_obj = Horizons(id=satellite_id, location='@10', epochs=epochs)
        # Get Cartesian positions in heliocentric ecliptic coordinate system
        helio_eph = helio_obj.vectors()
        
        # Create Horizons object - relative to parent planet center
        parent_obj = Horizons(id=satellite_id, location=f'@{parent_id}', epochs=epochs)
        # Get Cartesian positions relative to parent planet
        parent_eph = parent_obj.vectors()
        
        # Process heliocentric data
        df_xyz = pd.DataFrame({
            'date': helio_eph['datetime_str'],  # Date
            'x': helio_eph['x'],   # X component of position vector, AU
            'y': helio_eph['y'],
            'z': helio_eph['z'],
            'vx': helio_eph['vx'],  # X component of velocity vector (au/day)
            'vy': helio_eph['vy'],
            'vz': helio_eph['vz'],
            'lighttime': helio_eph['lighttime'],  # One-way down-leg Newtonian light-time, in days
            'range': helio_eph['range'],  # Distance from coordinate center (au)
            'range_rate': helio_eph['range_rate'] # Radial velocity relative to coordinate center (au/day)
        })
        
        # Process data relative to parent planet
        df_rel_xyz = pd.DataFrame({
            'date': parent_eph['datetime_str'],  # Date
            'rel_x': parent_eph['x'],   # X component of relative position vector, AU
            'rel_y': parent_eph['y'],
            'rel_z': parent_eph['z'],
            'rel_vx': parent_eph['vx'],  # X component of relative velocity vector (au/day)
            'rel_vy': parent_eph['vy'],
            'rel_vz': parent_eph['vz'],
            'rel_lighttime': parent_eph['lighttime'],  # One-way down-leg Newtonian light-time, in days
            'rel_range': parent_eph['range'],  # Distance from parent planet center (au)
            'rel_range_rate': parent_eph['range_rate'] # Radial velocity relative to parent planet center (au/day)
        })
        
        # Calculate ecliptic coordinates
        ecliptic_data = []
        for i, row in df_xyz.iterrows():
            # Convert Cartesian coordinates to ecliptic coordinates
            ecl_lon, ecl_lat = convert_xyz_to_ecliptic(row['x'], row['y'], row['z'])
            ecliptic_data.append({
                'date': row['date'],
                f'{satellite_id}_lon': ecl_lon,  # Ecliptic longitude (degrees)
                f'{satellite_id}_lat': ecl_lat,   # Ecliptic latitude (degrees)
                f'{satellite_id}_r': row['range']     # Distance (AU)
            })
        
        # Create ecliptic coordinate DataFrame
        df_ecliptic = pd.DataFrame(ecliptic_data)
        
        # Save as CSV files
        df_xyz.to_csv(xyz_filename, index=False)
        df_rel_xyz.to_csv(rel_xyz_filename, index=False)
        df_ecliptic.to_csv(ecliptic_filename, index=False)
        
        print(f'Satellite {satellite_id} query xyz, relative distance to parent planet, and ecliptic coordinates finished, files saved')
        return True
        
    except Exception as e:
        print(f"Error processing satellite {satellite_id}: {e}")
        return False

# Prepare directories for saving files
xyz_dir = '../../data/00_raw/helio_cart_states_00h/satellites_daily_1849/'
rel_xyz_dir = '../../data/00_raw/plnt_cart_state_00h/'
ecliptic_dir = '../../data/00_raw/helio_ecl_sph_00h/satellites_daily_1849/'

# Ensure directories exist
os.makedirs(xyz_dir, exist_ok=True)
os.makedirs(rel_xyz_dir, exist_ok=True)
os.makedirs(ecliptic_dir, exist_ok=True)

# Process different groups of satellite data
satellite_groups = [
    (s0, s0_date[0], s0_date[1], "Normal satellite group"),
    (st635, st635_date[0], st635_date[1], "Special satellite 635"),
    (st6, st6_date[0], st6_date[1], "Special satellite group 6"),
    (st7, st7_date[0], st7_date[1], "Special satellite group 7")
]

# Iterate through and process all satellite groups
for satellites, start, end, group_name in satellite_groups:
    print(f"\nStarting to process {group_name}, date range: {start} to {end}")
    
    for satellite_id in tqdm(satellites, desc=f"Processing {group_name}"):
        success = process_satellite_data(satellite_id, start, end, xyz_dir, rel_xyz_dir, ecliptic_dir)
        # Add a short delay to avoid high request frequency
        time.sleep(2 if success else 5)  # If an error occurs, wait longer before retrying

print('All tasks completed')

In [ ]:
# Batch Processing Asteroids

from astroquery.jplhorizons import Horizons
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, CartesianRepresentation
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy import units as u
import time
import os
from tqdm import tqdm

def convert_xyz_to_ecliptic(x, y, z):
    """ Use AstroPy to convert Cartesian coordinates to ecliptic longitude, latitude, and distance """
    # Create a celestial coordinate object, coordinates in astronomical units (AU)
    cart_coord = CartesianRepresentation(x * u.au, y * u.au, z * u.au)
    # Convert Cartesian coordinates to ecliptic coordinate system, specify J2000 reference epoch (default)
    sky_coord = SkyCoord(cart_coord, frame=HeliocentricMeanEcliptic(equinox='J2000'))
    # Return ecliptic longitude and latitude in degrees
    ecl_lon = sky_coord.lon.wrap_at(360 * u.deg).degree  # Normalize to 0-360 degrees
    ecl_lat = sky_coord.lat.degree
    # Calculate distance directly from Cartesian coordinates to avoid unit conversion issues
    distance = np.sqrt(x**2 + y**2 + z**2)
    return ecl_lon, ecl_lat, distance

# Prepare directories for saving files
xyz_dir = '../../data/00_raw/helio_cart_states_00h/asteroids_daily_1849/'
lon_lat_dir = '../../data/00_raw/helio_ecl_sph_00h/asteroids_daily_1849/'

# Ensure directories exist
os.makedirs(xyz_dir, exist_ok=True)
os.makedirs(lon_lat_dir, exist_ok=True)

# List of celestial bodies
# Note: Ensure sbdb_10km.csv is in the correct path
try:
    df_id = pd.read_csv('../../data/00_raw/sbdb_10km.csv')
    asteroids0 = df_id[(df_id['diameter'] >= 50) & (df_id['diameter'] < 910)]['spkid'].tolist()
except FileNotFoundError:
    print("Warning: sbdb_10km.csv not found. Please check the file path.")
    asteroids0 = []

total_asteroids = len(asteroids0)
print(f"Total number of asteroids matching criteria: {total_asteroids}")

# Batch processing settings
batch_size = 80  # 80 asteroids per batch
num_batches = (total_asteroids + batch_size - 1) // batch_size  # Calculate number of batches (round up)

# Define time range
start_date = '1849-01-01'
end_date = '2051-01-01'
epochs = {'start': start_date, 'stop': end_date, 'step': '1d'}

# Process by batches
for batch_num in range(num_batches):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, total_asteroids)
    asteroids_batch = asteroids0[start_idx:end_idx]
    
    print(f"Processing batch {batch_num+1}/{num_batches}, containing {len(asteroids_batch)} asteroids")
    
    # Query for each asteroid in the current batch
    for planet_id in tqdm(asteroids_batch, desc=f"Batch {batch_num+1}"):
        try:
            # Set file paths
            xyz_filename = f'{xyz_dir}{planet_id}_xyz.csv'
            lon_lat_filename = f'{lon_lat_dir}{planet_id}_lon_lat.csv'
            
            # Skip if files already exist
            if os.path.exists(xyz_filename) and os.path.exists(lon_lat_filename):
                # print(f'Asteroid {planet_id} already exists, skipping')
                continue
                
            # Create Horizons object, query by SPK ID
            obj = Horizons(id=f'DES= {planet_id};', location='@10', epochs=epochs)
            # Get Cartesian positions in heliocentric ecliptic coordinate system
            eph = obj.vectors()
            
            # Convert to ecliptic coordinates
            ecl_lon, ecl_lat, distance = convert_xyz_to_ecliptic(eph['x'], eph['y'], eph['z'])
            eph['ecl_lon'] = ecl_lon
            eph['ecl_lat'] = ecl_lat
            eph['distance'] = distance
            
            # Create Cartesian coordinate DataFrame
            df_xyz = pd.DataFrame({
                'date': eph['datetime_str'],  # Date
                'x': eph['x'],   # X component of position vector, AU
                'y': eph['y'],
                'z': eph['z'],
                'vx': eph['vx'],  # X component of velocity vector (au/day)
                'vy': eph['vy'],
                'vz': eph['vz'],
                'lighttime': eph['lighttime'],  # One-way down-leg Newtonian light-time, in days
                'range': eph['range'],  # Distance from coordinate center (au)
                'range_rate': eph['range_rate'] # Radial velocity relative to coordinate center (au/day)
            })
            
            # Create a DataFrame to store ecliptic longitude and latitude data
            df_lon_lat = pd.DataFrame({
                'date': eph['datetime_str'],
                f'{planet_id}_lon': eph['ecl_lon'],
                f'{planet_id}_lat': eph['ecl_lat'],
                f'{planet_id}_r': eph['distance']
            })
            
            # Save as CSV files
            df_xyz.to_csv(xyz_filename, index=False)
            df_lon_lat.to_csv(lon_lat_filename, index=False)
            
            # print(f'{planet_id} query xyz, convert to ecliptic finished, files saved')
            
            # Short pause to avoid frequent requests to the server
            time.sleep(2)
            
        except Exception as e:
            print(f"Error processing asteroid ID {planet_id}: {str(e)}")
            # Log error information to file
            with open('error_log.txt', 'a') as f:
                f.write(f"Asteroid ID {planet_id} processing failed: {str(e)}\n")
    
    print(f"Batch {batch_num+1} completed")
    
    # If not the last batch, pause for a while
    if batch_num < num_batches - 1:
        pause_time = 60  # Pause for 60 seconds
        print(f"Pausing for {pause_time} seconds before continuing to the next batch...")
        time.sleep(pause_time)

print('All batches processed')